In [26]:
import requests
from bs4 import BeautifulSoup as bs
import os
import time
import io
from random import randint
from time import sleep
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from PIL import Image 
from selenium.webdriver.common.action_chains import ActionChains
import requests
from bs4 import BeautifulSoup as bs
import os
import time
import io
from random import randint
from time import sleep
import re
import datetime
import time


headers = {

'Host':'search.anccnet.com',
'Referer':'http://search.anccnet.com/searchResult2.aspx?keyword=6901668054013',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.84 Safari/537.36',
'Upgrade-Insecure-Requests':'1',
'Accept-Encoding':'gzip, deflate',
    
'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',

'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.5,en;q=0.3',

'DNT':'1',
'Connection':'keep-alive',
 'Cookies':'ASP.NET_SessionId=3fwuki45cxn12a45k3tsbh45'
    
}


def get_same_question(link,headers,proxy,cookie):
    res = requests.get(link,headers=headers,proxies = proxy, timeout = 20,cookies = cookie)
    res.encoding ='gbk'
    soup =bs(res.text, "lxml")
    if '401' in soup.select('title')[0].text :
        proxy = get_new_ip()
        print(proxy)
        
        res = requests.get(link,headers=headers,proxies = proxy, timeout = 20 ,cookies = cookie)
        res.encoding= 'gbk'
        soup =bs(res.content, "lxml")
    # sleep(randint(1,5))

     
    return soup,proxy


def get_soup(mystring):
    soup =bs(mystring)
    return

def get_product_info(new_res):
    mydic ={}
    mydic['product_price'] = 'null'
    for line in new_res.select('script')[-2].text.split('\n'):   
        if 'txt_gtin' in line :
            m = re.search(r"src =(.*)\/\>",line)
            mydic['product_pic'] = m.group(1)
        if 'Att_Sys_zh-cn_141_G' in line :   
            mydic['product_name'] = re.search(r",\'(.*)\'",line).group(1) 
        if 'Att_Sys_zh-cn_74_G' in line :   
            mydic['product_place'] = re.search(r",\'(.*)\'",line).group(1)  
        if 'Att_Sys_zh-cn_22_G' in line :   
            mydic['product_UNSPSC'] = re.search(r",\'(.*)\'",line).group(1)     
        if 'Att_Sys_zh-cn_332_G' in line :   
            mydic['product_package'] = re.search(r",\'(.*)\'",line).group(1)          
        if 'Att_Sys_zh-cn_196_G' in line :   
            mydic['product_price'] = re.search(r",\'(.*)\'",line).group(1)
    return mydic

def get_new_ip ():
    ip = requests.get("http://http.tiqu.qingjuhe.cn/getip3?num=1&type=1&pro=&city=0&yys=0&port=1&pack=23882&ts=0&ys=0&cs=0&lb=1&sb=0&pb=4&mr=0&regions=")
    proxyDict = { 
              "http"  : "http://"+ip.text.strip(), 
              "https" : "https://"+ip.text.strip()
            }
    return proxyDict

# def get_new_ip ():
#     rand_ip = randint(1,len(ip_pools))
#     temp_ip = {'http':ip_pools[rand_ip]}
#     return temp_ip

In [2]:


def get_cach(driver):
    while True:
        driver.get('http://search.anccnet.com/searchResult2.aspx')   
        driver.fullscreen_window()
        sleep(2)

        element = driver.find_element_by_class_name("l-captcha")
        element.click()
        sleep(2)
        driver.save_screenshot('tem.png',)

        #######
        im = Image.open('tem.png') 


        im = im.resize((1440,900))

        # 图片的宽度和高度
        img_size = im.size
        print('图片识别')

        x = 720
        y = 154
        w = 300
        h = 160
        test = im.crop((x, y, x+w, y+h))
        width = test.size[0]
        height = test.size[1]
        test = test.convert('RGB')
        array = []
        for x in range(width):
            for y in range(height):
                r, g, b = test.getpixel((x,y))
                rgb = (r, g, b)
                if sum(rgb) == 765:
                    array.append((x,y))
        x_set = list()
        y_set = list()
        for line in array:
            x_set.append(line[0])
            y_set.append(line[1])
        my_x = int(sum(x_set) / float(len(x_set)))
        my_y = int(sum(y_set) / float(len(y_set)))
    #     print(my_x,my_y)
        test.load()[my_x,my_y] = (0,0,0)


        element = driver.find_elements_by_xpath("//iframe")[1]
        print(element.location)

        sleep(1)
        action = ActionChains(driver)
        action.move_to_element_with_offset( element, my_x,  my_y)
        # sleep(1)
        action.click()
    #     sleep(1)
#         action.click()
        action.perform()

        sleep(10)
        driver.refresh()
        res = bs(driver.page_source)
        if len(res.select('#Label1')) != 0:
            print('验证成功')
            break
        else:
            print('验证失败')
            driver.refresh()
            continue
    return driver,test


def  create_driver():
    option = webdriver.ChromeOptions()
    option.arguments.clear()
    for key in headers:
        mystring = key+'=\"'+headers[key]+"\""
        option.add_argument(mystring)
    option.add_argument('--headless')
    ip =  get_new_ip()['http']
    my_proxy ='--proxy-server=' + ip
    
    option.add_argument(my_proxy) 
    option.add_argument("--window-size=1440,900")
    driver = webdriver.Chrome('../test/chromedriver',chrome_options=option)
    driver.set_window_rect(0,0,1440,900)
    driver.delete_all_cookies()

    
    return driver,option,ip


In [3]:
import openpyxl
import pandas

# myexcel = pandas.read_excel('test.xlsx',names = ['index','UNSPSC_code','product','package','unit','place'] )
myexcel = pandas.read_excel('test.xlsx',names=['index','product_id','product_name','package','unit','place'])

In [4]:
link_list = []
for  line in myexcel[112245:200000].iterrows():
    link_list.append(line[1]['product_id'])
    
#     try:
#         link_list.append(int(line[1]['条码']))
#     except:
#         continue
# len(link_list)

len(link_list)

87755

In [12]:
proxy

{'http': 'http://220.186.190.174:4261',
 'https': 'https://220.186.190.174:4261'}

In [6]:
###############
result = {}###########3
##########
ip_pools = []


In [27]:

proxy = get_new_ip ()

cookie = {'ASP.NET_SessionId':'3fwuki45cxn12a45k3tsbh45','cuuid_322149':'e9ffefb10e56145bc00f80a786c85530'}
proxy

{'http': 'http://58.218.92.129:27720', 'https': 'https://58.218.92.129:27720'}

In [28]:


for index in range(83316,len(link_list)):
#     row = myexcel.iloc[index]
    
    temp_list = []
    mylink = "http://search.anccnet.com/searchResult2.aspx?keyword={}"
#     link = mylink.format(row['UNSPSC_code'])
    key = link_list[index]
    link = mylink.format(key)
    
    
    while True:
        try:
            res,proxy = get_same_question(link,headers,proxy,cookie) 
            if len(res.select('#Label4')) != 0:
                print('请验证')
                driver,option,ip = create_driver()
                driver,test = get_cach(driver)
                cookie['ASP.NET_SessionId'] = driver.get_cookies()[0]['value']
                driver.close()                
                res,proxy = get_same_question(link,headers,proxy,cookie) 
            break
        except requests.exceptions.Timeout:
            result[key] = "找不到"   
            print('找不到 why???? time out' )
            proxy = get_new_ip()
            break
        except :
            proxy = get_new_ip()
            print('change ip')
            continue
            
#     sleep(randint(1,2))
    headers['Referer'] = link
    print(index,headers['Referer'])


        
    if   len(res.select('#Label1')) == 0 :
        result[key] = "找不到" 
        print("找不到" )
        continue
    if  '已下市' in res.select('#Label1')[0].text :
        result[key] = "已下市"  
        print("已下市" )
        continue
    if '暂无相关信息' in res.select('#Label1')[0].text :
        result[key] = "暂无相关信息" 
        print("暂无相关信息" )
        continue
    if len(res.select('.p-info a')) == 0:
        result[key] = "奇怪了" 
        print("奇怪了" )
        continue
    if "未公开" in (res.select('.p-supplier dd')[-1].text):
        result[key] = "未公开" 
        print("未公开")
        continue
    info_link = res.select('.p-info a')[0]['href']
    
    while True:
        try:
            new_res,proxy = get_same_question(info_link,headers,proxy,cookie) 
            break
        except requests.exceptions.Timeout:
            proxy = get_new_ip()
            result[key] = "time out"  
            
            print('进入商品找不到，time out')
            break
        except :
            proxy = get_new_ip()
            continue
            
 
       
    try:    
        temp_dic = get_product_info(new_res)
    except:
        temp_dic = {}
    temp_dic_len = len(temp_dic)
    temp_dic['link'] = res.select('.p-info a')[0].text
    temp_dic['brand'] = res.select('.p-supplier dd')[0].text
    temp_dic['company'] = res.select('.p-supplier dd')[1].text
    result[res.select('.p-info a')[0].text] = temp_dic
    print(index,"正确资料",temp_dic_len)
    ip_pools.append(proxy['http'])

83316 http://search.anccnet.com/searchResult2.aspx?keyword=6916999000127
暂无相关信息
83317 http://search.anccnet.com/searchResult2.aspx?keyword=6928913480105
83317 正确资料 6
83318 http://search.anccnet.com/searchResult2.aspx?keyword=6930646925310
奇怪了
83319 http://search.anccnet.com/searchResult2.aspx?keyword=6930371918021
83319 正确资料 6
83320 http://search.anccnet.com/searchResult2.aspx?keyword=6916836888290
83320 正确资料 6
83321 http://search.anccnet.com/searchResult2.aspx?keyword=6937412826055
暂无相关信息
83322 http://search.anccnet.com/searchResult2.aspx?keyword=6931166360049
83322 正确资料 6
83323 http://search.anccnet.com/searchResult2.aspx?keyword=6901923043867
83323 正确资料 6
83324 http://search.anccnet.com/searchResult2.aspx?keyword=6924051120477
暂无相关信息
83325 http://search.anccnet.com/searchResult2.aspx?keyword=6920148672348
暂无相关信息
83326 http://search.anccnet.com/searchResult2.aspx?keyword=6921079004963
已下市
83327 http://search.anccnet.com/searchResult2.aspx?keyword=5000386004628
暂无相关信息
83328 http://sea

83411 http://search.anccnet.com/searchResult2.aspx?keyword=6901101242389
83411 正确资料 6
83412 http://search.anccnet.com/searchResult2.aspx?keyword=6920367166833
已下市
83413 http://search.anccnet.com/searchResult2.aspx?keyword=6902635400221
83413 正确资料 6
83414 http://search.anccnet.com/searchResult2.aspx?keyword=6936238480168
暂无相关信息
83415 http://search.anccnet.com/searchResult2.aspx?keyword=8887549104879
暂无相关信息
83416 http://search.anccnet.com/searchResult2.aspx?keyword=6924216661036
奇怪了
83417 http://search.anccnet.com/searchResult2.aspx?keyword=6934668733533
83417 正确资料 6
83418 http://search.anccnet.com/searchResult2.aspx?keyword=6926512000427
暂无相关信息
83419 http://search.anccnet.com/searchResult2.aspx?keyword=6918404582326
奇怪了
83420 http://search.anccnet.com/searchResult2.aspx?keyword=6918551807525
奇怪了
83421 http://search.anccnet.com/searchResult2.aspx?keyword=6931830494711
暂无相关信息
83422 http://search.anccnet.com/searchResult2.aspx?keyword=6920865299194
奇怪了
83423 http://search.anccnet.com/searc

83506 正确资料 6
83507 http://search.anccnet.com/searchResult2.aspx?keyword=6923059762245
已下市
83508 http://search.anccnet.com/searchResult2.aspx?keyword=6928001891882
已下市
83509 http://search.anccnet.com/searchResult2.aspx?keyword=6920368440123
已下市
找不到 why???? time out
83510 http://search.anccnet.com/searchResult2.aspx?keyword=6926329000702
已下市
找不到 why???? time out
83511 http://search.anccnet.com/searchResult2.aspx?keyword=6928491990102
已下市
找不到 why???? time out
83512 http://search.anccnet.com/searchResult2.aspx?keyword=6922629703039
已下市
83513 http://search.anccnet.com/searchResult2.aspx?keyword=6930410833049
83513 正确资料 6
83514 http://search.anccnet.com/searchResult2.aspx?keyword=6936027263828
暂无相关信息
83515 http://search.anccnet.com/searchResult2.aspx?keyword=6924357839066
83515 正确资料 6
83516 http://search.anccnet.com/searchResult2.aspx?keyword=6936380660623
83516 正确资料 6
83517 http://search.anccnet.com/searchResult2.aspx?keyword=6924357838502
83517 正确资料 6
83518 http://search.anccnet.com/search

83602 http://search.anccnet.com/searchResult2.aspx?keyword=6925151300073
暂无相关信息
83603 http://search.anccnet.com/searchResult2.aspx?keyword=6920367243343
已下市
83604 http://search.anccnet.com/searchResult2.aspx?keyword=6927727908010
奇怪了
83605 http://search.anccnet.com/searchResult2.aspx?keyword=6924096120333
{'http': 'http://58.218.92.126:28402', 'https': 'https://58.218.92.126:28402'}
83605 正确资料 6
83606 http://search.anccnet.com/searchResult2.aspx?keyword=6922950120086
暂无相关信息
83607 http://search.anccnet.com/searchResult2.aspx?keyword=6903071663034
83607 正确资料 6
83608 http://search.anccnet.com/searchResult2.aspx?keyword=6920368430285
已下市
83609 http://search.anccnet.com/searchResult2.aspx?keyword=6923659210467
83609 正确资料 6
找不到 why???? time out
83610 http://search.anccnet.com/searchResult2.aspx?keyword=6924956410376
83610 正确资料 6
83611 http://search.anccnet.com/searchResult2.aspx?keyword=6924216688088
奇怪了
83612 http://search.anccnet.com/searchResult2.aspx?keyword=6924832920081
已下市
83613 http:

83697 正确资料 6
83698 http://search.anccnet.com/searchResult2.aspx?keyword=6940520700674
83698 正确资料 6
83699 http://search.anccnet.com/searchResult2.aspx?keyword=6922862489882
已下市
找不到 why???? time out
83700 http://search.anccnet.com/searchResult2.aspx?keyword=4893153384785
已下市
找不到 why???? time out
83701 http://search.anccnet.com/searchResult2.aspx?keyword=4893773710124
已下市
找不到 why???? time out
83702 http://search.anccnet.com/searchResult2.aspx?keyword=4893773810114
已下市
找不到 why???? time out
83703 http://search.anccnet.com/searchResult2.aspx?keyword=4897003019834
已下市
找不到 why???? time out
83704 http://search.anccnet.com/searchResult2.aspx?keyword=4909411012052
已下市
找不到 why???? time out
83705 http://search.anccnet.com/searchResult2.aspx?keyword=4911691473355
已下市
83706 http://search.anccnet.com/searchResult2.aspx?keyword=4971710408713
暂无相关信息
83707 http://search.anccnet.com/searchResult2.aspx?keyword=4972590109516
暂无相关信息
83708 http://search.anccnet.com/searchResult2.aspx?keyword=6228184200113
暂无相

...

进入商品找不到，time out
87299 正确资料 6
找不到 why???? time out
87300 http://search.anccnet.com/searchResult2.aspx?keyword=6911445201348
进入商品找不到，time out
87300 正确资料 6
87301 http://search.anccnet.com/searchResult2.aspx?keyword=6937997906111
87301 正确资料 6
87302 http://search.anccnet.com/searchResult2.aspx?keyword=6932953900264
87302 正确资料 6
87303 http://search.anccnet.com/searchResult2.aspx?keyword=6940520700513
87303 正确资料 6
87304 http://search.anccnet.com/searchResult2.aspx?keyword=6935092100427
暂无相关信息
87305 http://search.anccnet.com/searchResult2.aspx?keyword=6928548200130
奇怪了
87306 http://search.anccnet.com/searchResult2.aspx?keyword=6934504220043
暂无相关信息
87307 http://search.anccnet.com/searchResult2.aspx?keyword=6926365800151
已下市
87308 http://search.anccnet.com/searchResult2.aspx?keyword=6941735900637
暂无相关信息
87309 http://search.anccnet.com/searchResult2.aspx?keyword=6934956117116
暂无相关信息
87310 http://search.anccnet.com/searchResult2.aspx?keyword=6935371700065
暂无相关信息
87311 http://search.anccnet.com/se

87396 正确资料 6
87397 http://search.anccnet.com/searchResult2.aspx?keyword=6930694689004
87397 正确资料 6
87398 http://search.anccnet.com/searchResult2.aspx?keyword=6923826620044
已下市
87399 http://search.anccnet.com/searchResult2.aspx?keyword=6930883618365
暂无相关信息
87400 http://search.anccnet.com/searchResult2.aspx?keyword=6924517601120
奇怪了
87401 http://search.anccnet.com/searchResult2.aspx?keyword=6936397016147
暂无相关信息
87402 http://search.anccnet.com/searchResult2.aspx?keyword=6916954020299
暂无相关信息
87403 http://search.anccnet.com/searchResult2.aspx?keyword=6923767320560
暂无相关信息
87404 http://search.anccnet.com/searchResult2.aspx?keyword=6911575891082
已下市
87405 http://search.anccnet.com/searchResult2.aspx?keyword=6921458682409
奇怪了
87406 http://search.anccnet.com/searchResult2.aspx?keyword=6921296204252
87406 正确资料 6
87407 http://search.anccnet.com/searchResult2.aspx?keyword=6915224346091
奇怪了
87408 http://search.anccnet.com/searchResult2.aspx?keyword=6917608133235
暂无相关信息
87409 http://search.anccnet.co

87497 http://search.anccnet.com/searchResult2.aspx?keyword=6930679201726
87497 正确资料 6
87498 http://search.anccnet.com/searchResult2.aspx?keyword=6923090198195
87498 正确资料 6
87499 http://search.anccnet.com/searchResult2.aspx?keyword=6926091503265
已下市
87500 http://search.anccnet.com/searchResult2.aspx?keyword=6930679219066
暂无相关信息
87501 http://search.anccnet.com/searchResult2.aspx?keyword=6920802803064
奇怪了
87502 http://search.anccnet.com/searchResult2.aspx?keyword=6901923033813
87502 正确资料 6
87503 http://search.anccnet.com/searchResult2.aspx?keyword=6935915400352
奇怪了
87504 http://search.anccnet.com/searchResult2.aspx?keyword=6917121680285
已下市
87505 http://search.anccnet.com/searchResult2.aspx?keyword=6930918736408
暂无相关信息
{'http': 'http://58.218.198.159:28620', 'https': 'https://58.218.198.159:28620'}
87506 http://search.anccnet.com/searchResult2.aspx?keyword=6925303718787
奇怪了
87507 http://search.anccnet.com/searchResult2.aspx?keyword=6901295365062
87507 正确资料 6
87508 http://search.anccnet.co

87590 正确资料 6
87591 http://search.anccnet.com/searchResult2.aspx?keyword=6928573330147
87591 正确资料 6
87592 http://search.anccnet.com/searchResult2.aspx?keyword=6928573330178
87592 正确资料 6
87593 http://search.anccnet.com/searchResult2.aspx?keyword=6928812500201
已下市
87594 http://search.anccnet.com/searchResult2.aspx?keyword=6927654200225
奇怪了
87595 http://search.anccnet.com/searchResult2.aspx?keyword=6921078503474
{'http': 'http://58.218.198.160:25420', 'https': 'https://58.218.198.160:25420'}
87595 正确资料 6
87596 http://search.anccnet.com/searchResult2.aspx?keyword=6925993550186
87596 正确资料 6
87597 http://search.anccnet.com/searchResult2.aspx?keyword=6922872790695
已下市
87598 http://search.anccnet.com/searchResult2.aspx?keyword=6927209902154
暂无相关信息
87599 http://search.anccnet.com/searchResult2.aspx?keyword=6932794413923
87599 正确资料 6
87600 http://search.anccnet.com/searchResult2.aspx?keyword=6915878201272
已下市
87601 http://search.anccnet.com/searchResult2.aspx?keyword=6922896302065
暂无相关信息
87602 ht

87683 正确资料 6
87684 http://search.anccnet.com/searchResult2.aspx?keyword=6928913441953
87684 正确资料 6
87685 http://search.anccnet.com/searchResult2.aspx?keyword=6903193002391
87685 正确资料 6
87686 http://search.anccnet.com/searchResult2.aspx?keyword=6907259000376
暂无相关信息
87687 http://search.anccnet.com/searchResult2.aspx?keyword=6928370630211
暂无相关信息
87688 http://search.anccnet.com/searchResult2.aspx?keyword=6925260691024
已下市
87689 http://search.anccnet.com/searchResult2.aspx?keyword=6921079004109
已下市
87690 http://search.anccnet.com/searchResult2.aspx?keyword=6935915400239
奇怪了
87691 http://search.anccnet.com/searchResult2.aspx?keyword=6939949704141
87691 正确资料 6
87692 http://search.anccnet.com/searchResult2.aspx?keyword=6924452350299
已下市
87693 http://search.anccnet.com/searchResult2.aspx?keyword=6925384356977
奇怪了
87694 http://search.anccnet.com/searchResult2.aspx?keyword=6901923034094
87694 正确资料 6
87695 http://search.anccnet.com/searchResult2.aspx?keyword=6936509504043
暂无相关信息
87696 http://searc

In [ ]:
res,proxy = get_same_question(link,headers,proxy,cookie) 

In [ ]:
driver,option,ip = create_driver()
driver,test = get_cach(driver)

In [ ]:
link

In [ ]:
if '暂无相关信息' in new_res.select('#Label1')[0].text :
    print(123)

In [ ]:
k = 0
result_tmp = result.copy()
for line in result:
    if isinstance(result[line],str) :
        del result_tmp[line]
    if result[line] != '找不到':
        print(result[line],line)
    

print(k)

In [ ]:
result[6901010113169]

In [ ]:
code_list = pd.read_csv('编码最终版.csv')

segment_dic ={}
family_dic = {}
class_dic = {}
brick_dic = {}
for line in code_list.iterrows():
    segment_dic [line[1]['segment_code']] = line[1]["segment_name"]
    family_dic [line[1]['family_code']] = line[1]["family_name"]
    class_dic [line[1]['class_code']] = line[1]["class_name"]
    brick_dic [line[1]['brick_code']] = line[1]["brick_name"]
    

In [ ]:
output = open('编码最终版.txt','r',encoding = 'utf8')
encod_dic = {}
for line in output:
    myans = (line.split('!@#'))
    encod_dic[myans[-2]] = myans
output.close()    

In [ ]:
print(line)

In [ ]:
import pandas as pd
data = []

for key in result_tmp:
    if isinstance(result_tmp[key],str) :
        continue
    if  result_tmp[key] is None:
        continue
    
    tmp = []
    brand = result_tmp[key]['brand'].strip()
    company = result_tmp[key]['company'].strip()
    link = result_tmp[key]['link'].strip()
    product_UNSPSC = result_tmp[key]['product_UNSPSC'].strip()
    product_name = result_tmp[key]['product_name'].strip()
    product_package = result_tmp[key]['product_package'].strip()
    product_pic = result_tmp[key]['product_pic'].strip()
    product_place = result_tmp[key]['product_place'].strip()
    try:
        product_UNSPSC_code = product_UNSPSC.split('&')[0] 
        tmp = [brand,company,link,product_UNSPSC,product_name,product_package,product_place,product_UNSPSC_code]
        if product_UNSPSC_code in encod_dic:
            tmp.extend(encod_dic[product_UNSPSC_code])
#             print(tmp)
    except:
        tmp = [brand,company,link,product_UNSPSC,product_name,product_package,product_place,'null','null','null','null','null','null','null','null','null']
        data.append(tmp)
        continue           
    data.append(tmp)
#     break
    
df = pd.DataFrame(data,columns=['brand','company','link','product_UNSPSC','product_name','product_package','product_place','product_UNSPSC_code','segment_code','segment_name','family_code','family_name','class_code','class_name','brick_code','brick_name'])

In [ ]:
df.to_excel('苏州酸奶.xlsx',encoding='utf-8',index= False)

In [29]:
import json

with open('全部商品_112245_200000.json', 'w') as fp:
    json.dump(result, fp)

In [ ]:
    rand_ip = randint(1,len(ip_pools))
    temp_ip = {'http':'ip_pools[rand_ip]'}
return temp_ip

In [ ]:
df